## Imports

In [43]:
import json
import urllib.request
import urllib.parse
import plotly.express as px
import plotly.io as pio
from sentence_transformers import CrossEncoder
from sentence_transformers import SentenceTransformer
import torch
from neo4j import GraphDatabase
from matplotlib import pyplot
import seaborn as sns
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.metrics import classification_report
tqdm.pandas()
%matplotlib inline


model1 = SentenceTransformer('lighteternal/stsb-xlm-r-greek-transfer')

model2 = CrossEncoder('lighteternal/nli-xlm-r-greek')

/home/earendil/anaconda3/envs/phd/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Loading Greek FEVER subset

In [15]:
df_gr = pd.pandas.read_excel(
    "greek_fever_benchmark.xlsx", engine='openpyxl')

In [ ]:
df_gr = df_gr.replace('\n',' ', regex=True)
df_gr = df_gr.replace('&nbsp;',' ', regex=True)
df_gr = df_gr.replace('&amp;',' ', regex=True)
print(len(df_gr))


df_gr["evidence_tok"] =  df_gr['evidence_gr'].progress_apply(lambda x: sent_tokenize(x) if pd.isna(x)==False else '-')
df_gr

In [18]:
df_gr_tok = df_gr.explode('evidence_tok')


In [8]:
df_gr_tok.to_csv("./fever_dataset_tok.csv", sep = ',')

## Populating the Neo4j with Wikipedia data

In [9]:
driver = GraphDatabase.driver(
    'bolt://localhost:11005', auth=('neo4j', 'password'))


def run_query(query, params={}):
    with driver.session(database='fevergr') as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
run_query("""

    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-dd090feb-9867-4887-9be2-93af8aeec75f/fever_dataset_tok.csv' AS row
    MERGE (a:Article {title: coalesce(row.Column1, ' '), maintext: row.evidence_gr})
    WITH a, row
    UNWIND row.evidence_tok AS section
    WITH a, section
    MERGE (s:Section {name: section})
    MERGE (a)-[r:HAS_SECTION]->(s)

    """)

In [ ]:
run_query(
    "CREATE CONSTRAINT IF NOT EXISTS ON (e:Entity) ASSERT e.wikiDataItemId is UNIQUE;")

In [ ]:

user_key = "YOUR JSI WIKIFIER API KEY"

run_query("""
CALL apoc.periodic.iterate('
 MATCH (s:Section) RETURN s
 ','
 WITH s, "http://wikifier.org/annotate-article?" +
        "text=" + apoc.text.urlencode(s.name) + "&" +
        "lang=auto&" +
        "pageRankSqThreshold=0.99&" +
        "applyPageRankSqThreshold=true&" +
        "maxMentionEntropy=3&" +
        "wikiDataClasses=false&" +
        "wikiDataClassIds=false&" +
        "userKey=" + $userKey as url
CALL apoc.load.json(url) YIELD value
UNWIND value.annotations as annotation
MERGE (e:Entity{wikiDataItemId:annotation.wikiDataItemId})
ON CREATE SET e.title = annotation.title, e.url = annotation.url
MERGE (s)-[:HAS_ENTITY]->(e)',
{batchSize:8, params: {userKey:$user_key}})
""", {"user_key": user_key})

In [ ]:
run_query("""
// Iterate over entities
MATCH (e:Entity)
// Prepare a SparQL query
WITH 'SELECT *
      WHERE{
        ?item rdfs:label ?name .
        filter (?item = wd:' + e.wikiDataItemId + ')
        filter (lang(?name) = "en" ) .
      OPTIONAL{
        ?item wdt:P31 [rdfs:label ?class] .
        filter (lang(?class)="en")
      }}' AS sparql, e
// make a request to Wikidata
CALL apoc.load.jsonParams(
    "https://query.wikidata.org/sparql?query=" + 
    apoc.text.urlencode(sparql),
     { Accept: "application/sparql-results+json"}, null)
YIELD value
UNWIND value['results']['bindings'] as row
FOREACH(ignoreme in case when row['class'] is not null then [1] else [] end | 
        MERGE (c:Class{name:row['class']['value']})
        MERGE (e)-[:INSTANCE_OF]->(c));    
""")

In [ ]:
run_query("""
MATCH (e:Entity)-[:INSTANCE_OF]->(c:Class)
WHERE c.name in ["άνθρωπος", "human"]
SET e:Person;
""")
run_query("""
MATCH (e:Entity)-[:INSTANCE_OF]->(c:Class)
WHERE c.name in ["επιχείρηση", "οργανισμός","εταιρεία", "business", "corporation", "organization", "company"]
SET e:Business;
""")

In [ ]:
run_query("""
MATCH (e:Business)
// Prepare a SparQL query
WITH 'SELECT *
      WHERE{
        ?item rdfs:label ?name .
        filter (?item = wd:' + e.wikiDataItemId + ')
        filter (lang(?name) = "auto" ) .
      OPTIONAL{
        ?item wdt:P452 [rdfs:label ?industry] .
        filter (lang(?industry)="auto")
      }}' AS sparql, e
// make a request to Wikidata
CALL apoc.load.jsonParams(
    "https://query.wikidata.org/sparql?query=" + 
    apoc.text.urlencode(sparql),
     { Accept: "application/sparql-results+json"}, null)
YIELD value
UNWIND value['results']['bindings'] as row
FOREACH(ignoreme in case when row['industry'] is not null then [1] else [] end | 
        MERGE (i:Industry{name:row['industry']['value']})
        MERGE (e)-[:PART_OF_INDUSTRY]->(i));
""")

In [ ]:
run_query("""
MATCH (e:Entity)-[:INSTANCE_OF]->(c:Class)
WHERE c.name in ["city", "πόλη"]
SET e:City;
""")

run_query("""
MATCH (e:Entity)-[:INSTANCE_OF]->(c:Class)
WHERE c.name in ["χώρα", "country"]
SET e:Country;
""")

## Helper functions

In [10]:
# Call Wikifier on hypothesis

# import urllib.parse, urllib.request, json

def CallWikifier(text, lang="el", threshold=0.99):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("auto", lang),
        ("secondaryAnnotLanguage", "en"),
        ("userKey", "unyifgroezsfzcbwhaprdfjhxskvgx"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "false"), ("ranges", "false"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=120) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    for annotation in response["annotations"]:
        print("%s (%s)" % (annotation["title"], annotation["wikiDataItemId"]))
    return response["annotations"]


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

### Running FEVER subset benchmark

In [ ]:
# Get all claims from greek_fever_beenchmark:
claims = df_gr['claim_gr']
claims.dropna(inplace=True)
print(f"Number of claims:{len(claims)}")
labels_farfetched = []

for claim in claims:
    # Wikify claim
    print('===============================')
    print(f'Processing claim: {claim}')
    hypo_list = CallWikifier(claim)

    # Shortest path between found entities in Neo4j graph
    hypo_ents = []
    print('Searching for entities in claim...')
    for item in hypo_list:
        hypo_ents.append(item["wikiDataItemId"])
        # print(item["wikiDataItemId"])
    if len(hypo_ents) > 0:
        path_count = (len(hypo_ents)-1)*2
        #print('Discovered entities in claim: ', hypo_ents)
        print('Checking chains of entity mentions, maximum possible length of path: ', path_count)
        
        # Evidence constructor on found entities
        query = """
        MATCH (n:Entity) WHERE n.wikiDataItemId IN $hypo_ents
        WITH collect(n) as nodes
        UNWIND nodes as n
        UNWIND nodes as m
        
        WITH nodes, n, m WHERE id(n) > id(m)
         MATCH path =( (n)-[*0..PATHCOUNT]-(m) ) WHERE NONE(k IN nodes(path) WHERE k:Class)
            WITH nodes, path WHERE ALL(n IN nodes WHERE n IN nodes(path))

        RETURN nodes(path), length(path) ORDER BY length(path) DESC LIMIT 1000
        """

        query_upd = (query.replace("PATHCOUNT", str(min(path_count, 3))))

        # shortestPath for multiple nodes:
        result = run_query(query_upd, {"hypo_ents": hypo_ents})
        
        # Also checking individual entity mentions
        print(f'Found {len(result)} chained entity mentions.')
        if len(result) < 1:
            print("Checking individual entities mentions...")
            query_nopath = """
                MATCH (n:Entity) WHERE n.wikiDataItemId IN $hypo_ents
                WITH collect(n) as nodes
                UNWIND nodes as n
                MATCH p=(n)<-[r:HAS_ENTITY]-(s:Section) 
                RETURN s.name
                """
            result = run_query(query_nopath, {"hypo_ents": hypo_ents})
            print(
                f"Found {len(result)} individual entity mentions on knowledge base")

        # Searching for premises with relevant entities
        query_upd = query
        candidate_premises = []
        temp = ""
        try:
            for path in result['nodes(path)']:
                for sent in path:
                    if sent['name'] is not None:
                        # print(sent['name'])
                        temp += sent['name']+' '
                candidate_premises.append(temp)
                temp = ""
                # print('--')

        except:
            for sent in result['s.name']:
                # print(sent)
                candidate_premises.append(sent)
                temp = ""
                # print('--')

        if len(candidate_premises) > 0:
            # Select best candidate evidence based on STS
            hypo_emb = model1.encode(claim, convert_to_tensor=True)
            cand_prem_emb = model1.encode(
                candidate_premises, convert_to_tensor=True)

            # Compute cosine-similarities
            from sentence_transformers import util
            cosine_scores = util.pytorch_cos_sim(hypo_emb, cand_prem_emb)
            # cosine_scores

            pd. set_option('display.max_columns', None)
            pd. set_option('display.max_rows', None)
            pd. set_option('display.max_colwidth', None)

            df = pd.DataFrame()
            df['candidates'] = candidate_premises
            df['similarities'] = pd.DataFrame(
                cosine_scores.cpu().numpy().transpose())
            df.drop_duplicates(inplace=True)
            df.sort_values(by='similarities', ascending=False)

            premise = candidate_premises[int(torch.argmax(cosine_scores))]

            # Most relevant premise comparison
            scores = model2.predict([(premise, claim)])

            print("\n Most relevant premise: ", premise)
            print("---")
            #print("Claim: ",claim)
            # Convert scores to labels
            label_mapping = ['REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO']
            labels = [label_mapping[score_max]
                      for score_max in scores.argmax(axis=1)]
            print("Score C/E/N: ", scores, labels)
            labels_farfetched.append("".join(labels))
        else:
            print('No relevant premises found in knowledge base! Not enough info to conduct claim verification based on existing data')
            labels_farfetched.append('NOT ENOUGH INFO')

    else:
        print('No entities matched in claim! Not enough info to conduct claim verification based on existing data')
        labels_farfetched.append('NOT ENOUGH INFO')

In [44]:
classification_report = classification_report(df_gr['true_result'], labels_farfetched)
print(classification_report)

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.36      0.80      0.49        20
        REFUTES       0.91      0.72      0.80        93
       SUPPORTS       0.84      0.70      0.76        37

       accuracy                           0.73       150
      macro avg       0.70      0.74      0.69       150
   weighted avg       0.82      0.73      0.75       150

